In [13]:
import jsonlines
import torchaudio
from pathlib import Path
import torch, random
import librosa, os
import IPython.display as ipd
from dotenv import load_dotenv
from spellchecker import SpellChecker
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
import pandas as pd
from sklearn.model_selection import train_test_split
import cv2
from collections import defaultdict
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
from skimage import io
from skimage.transform import resize
import matplotlib.pyplot as plt
import random
import matplotlib.patches as patches
import os

import torch
import torchvision
from torchvision import ops
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence

In [14]:
import jsonlines
import pandas as pd

data = {'image': [], 'caption': [], 'bbox': []}
load_dotenv()

TEAM_NAME = os.getenv("TEAM_NAME", "7up")
TEAM_TRACK = os.getenv("TEAM_TRACK", "advanced")


input_dir = Path(f"/home/jupyter/{TEAM_TRACK}")
with jsonlines.open(input_dir / "vlm.jsonl") as reader:
    for obj in reader:
        image = obj['image']
        annotations = obj['annotations']
        for annotation in annotations:
            caption = annotation['caption']
            bbox = annotation['bbox']
            data['image'].append(image)
            data['caption'].append(caption)
            data['bbox'].append(bbox)

df = pd.DataFrame(data)
grouped_df = df.groupby('image').agg(list).reset_index()
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
grouped_df.head()

,image,caption,bbox
0,image_0.jpg,"[grey missile, red, white, and blue light airc...","[[912, 164, 48, 152], [1032, 80, 24, 28], [704..."
1,image_1.jpg,"[grey camouflage fighter jet, grey and white f...","[[1112, 172, 64, 36], [1108, 512, 144, 48], [3..."
2,image_10.jpg,"[blue and grey fighter jet, blue helicopter, b...","[[792, 208, 52, 40], [964, 392, 52, 52], [408,..."
3,image_100.jpg,"[grey camouflage fighter jet, white, black, an...","[[1108, 224, 56, 44], [788, 148, 92, 32], [516..."
4,image_1000.jpg,"[red and white fighter plane, yellow commercia...","[[412, 252, 68, 36], [544, 276, 44, 36], [1092..."


In [15]:
unique_values = df['caption'].unique()
len(unique_values)

126

In [16]:
from torchvision import transforms

data_transforms = transforms.Compose([
    # transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

In [17]:
class ObjectDetectionDataset(Dataset):
    '''
    A Pytorch Dataset class to load the images and their corresponding annotations.
    
    Returns
    ------------
    images: torch.Tensor of size (B, C, H, W)
    gt bboxes: torch.Tensor of size (B, max_objects, 4)
    gt classes: torch.Tensor of size (B, max_objects)
    '''
    def __init__(self, df, img_dir, img_size, name2idx, transforms=None):
        self.image = df["image"]
        self.caption = df["caption"]
        self.bbox = df["bbox"]
        self.img_dir = img_dir
        self.img_size = img_size
        self.name2idx = name2idx
        self.transforms = transforms
        
    def __len__(self):
        return len(self.image)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.image[idx])
        img = Image.open(img_path).convert("RGB")
        
        if self.transforms:
            img = self.transforms(img)
        else:
            img = transforms.ToTensor()(img)
        
        gt_bboxes = torch.tensor(self.bbox[idx], dtype=torch.float32)
        gt_classes = torch.tensor([self.name2idx[name] for name in self.caption[idx]], dtype=torch.long)
        
        return img, gt_bboxes, gt_classes

In [18]:
img_width = 1520
img_height = 870
df_input = df
image_dir = "/home/jupyter/advanced/images/"
name2idx = {value: idx for idx, value in enumerate(unique_values)}
name2idx['pad'] = -1
idx2name = {v:k for k, v in name2idx.items()}

In [19]:
od_dataset = ObjectDetectionDataset(grouped_df, image_dir, (img_height, img_width), name2idx, data_transforms)


In [20]:
od_dataloader = DataLoader(od_dataset, batch_size=2)


In [21]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Function to display an image with bounding boxes and class labels
def show_image_with_boxes(image, bboxes, classes, class_names):
    fig, ax = plt.subplots(1)
    ax.imshow(image.permute(1, 2, 0))  # Convert CHW to HWC format for display

    for bbox, cls in zip(bboxes, classes):
        if bbox[0] == -1:
            break  # Skip padding bboxes

        # Unpack the bounding box
        x, y, width, height = bbox

        # Create a rectangle patch
        rect = patches.Rectangle((x, y), width, height, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)

        # Add the class label
        plt.text(x, y, class_names[int(cls)], color='yellow', fontsize=12, bbox=dict(facecolor='red', alpha=0.5))

    plt.show()

# Display some samples from the DataLoader
for batch_idx, (images, bboxes, classes) in enumerate(od_dataloader):
    if batch_idx >= 2:  # Display only the first 2 batches
        break

    for img_idx in range(images.size(0)):
        img = images[img_idx]
        bbox = bboxes[img_idx]
        cls = classes[img_idx]

        show_image_with_boxes(img, bbox, cls, name2idx.keys())

RuntimeError: stack expects each tensor to be equal size, but got [4, 4] at entry 0 and [6, 4] at entry 1

In [ ]:
for img_batch, gt_bboxes_batch, gt_classes_batch in od_dataloader:
    img_data_all = img_batch
    gt_bboxes_all = gt_bboxes_batch
    gt_classes_all = gt_classes_batch
    break
    
img_data_all = img_data_all[:2]
gt_bboxes_all = gt_bboxes_all[:2]
gt_classes_all = gt_classes_all[:2]

In [ ]:
# get class names
gt_class_1 = gt_classes_all[0].long()
gt_class_1 = [idx2name[idx.item()] for idx in gt_class_1]

gt_class_2 = gt_classes_all[1].long()
gt_class_2 = [idx2name[idx.item()] for idx in gt_class_2]

In [ ]:
nrows, ncols = (1, 2)
fig, axes = plt.subplots(nrows, ncols, figsize=(16, 8))

fig, axes = display_img(img_data_all, fig, axes)
fig, _ = display_bbox(gt_bboxes_all[0], fig, axes[0], classes=gt_class_1)
fig, _ = display_bbox(gt_bboxes_all[1], fig, axes[1], classes=gt_class_2)